In [6]:

from prepared_data.get_prepared_data import get_prepared_data
import pandas as pd
import numpy as np
import tensorflow as tf

from prepared_data.train_model import split_features_target_and_map_target, save_results, callbacks, columns_only_first_hist


In [7]:
train_org = pd.read_csv('data/train.csv')
train,_,_ = get_prepared_data(train_org,map_target=False)
# train = pd.read_csv('data/train_made.csv',index_col=0)


percent of object with nan value and orginals:  0.00, 85.915556


/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


percent of object with nan value and orginals:  0.00, 85.915556


In [40]:
# train_features, target = split_features_target_and_map_target(train)

# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(np.array(train_features))

# normalizer.trainable_variables
# normalizer(train_features)
binary_col = ['is_cup','_team_history_is_play_home_1','_team_history_is_cup_1','_team_history_coach_1','_team_history_target_1_-1.0', '_team_history_target_1_0.0', '_team_history_target_1_1.0']

binary_col_all = ['is_cup']
binary_col_all.extend([ home_or_away +i  for i in  binary_col[1:] for home_or_away in ['home','away'] ])
binary_col_all


['is_cup',
 'home_team_history_is_play_home_1',
 'away_team_history_is_play_home_1',
 'home_team_history_is_cup_1',
 'away_team_history_is_cup_1',
 'home_team_history_coach_1',
 'away_team_history_coach_1',
 'home_team_history_target_1_-1.0',
 'away_team_history_target_1_-1.0',
 'home_team_history_target_1_0.0',
 'away_team_history_target_1_0.0',
 'home_team_history_target_1_1.0',
 'away_team_history_target_1_1.0']

In [10]:

def build_and_compile_model(norm,n_neurons, dropout, learning_rate):
    model = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(n_neurons, activation= tf.keras.activations.relu),
    # tf.keras.layers.Dropout(dropout),

    tf.keras.layers.Dense(n_neurons, activation= tf.keras.activations.relu),

    tf.keras.layers.Dense(3)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model

def build_and_compile_model_1(shape,n_neurons, dropout, learning_rate):
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          16, activation='relu',
          input_shape=shape),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(3)
     ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model


In [41]:
from datetime import datetime
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from prepared_data.completed_missing_data import remove_history_col
import operator
from sklearn.model_selection import StratifiedKFold

n_neurons = 10
dropout = 0.0
learning_rate = 0.001
batch_size = None

train_h_8,_,_ = get_prepared_data(train_org, number_of_history_matches=4, map_target=False)



for n_history in range(3,4):


    # train = remove_history_col(train_h_8,number_of_history_matches=4)
    train_features, target = split_features_target_and_map_target(train)

    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)

    normalizer = tf.keras.layers.Input(shape=train_features.shape[1], name='input' )


    # normalizer.adapt(np.array(train_features))

    dnn_model = build_and_compile_model(normalizer,n_neurons,dropout,learning_rate)


    model_name = f'dnn_l_{n_neurons}_l_{n_neurons}_number_of_history_matches_{4}_pipe_' + datetime.now().strftime("%Y:%m:%d-%H:%M:%S")
    output_path = 'evaluate_results/'+ model_name


    save_results(output_path, f'lr={learning_rate} batch={batch_size}', dnn_model.summary)
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=33)

    i= 0
    max_value_val_acc = list(range(0,5))
    min_value_val_loss = list(range(0, 5))
    train_acc = list(range(0,5))
    train_loss = list(range(0,5))

    for train_index, test_index in skf.split(train_features,target):

        model_full_name =  model_name +  f'/folds_{i}'

        early_stop, reduce_lr, tensorboard_callback, checkpoint_callback =   callbacks(model_full_name)

        history = dnn_model.fit(
            train_features.iloc[train_index],
            target.iloc[train_index],
            validation_data = (train_features.iloc[test_index],target.iloc[test_index]),
            verbose=0, epochs=10,
           callbacks=[early_stop,checkpoint_callback, tensorboard_callback],
            batch_size = batch_size
        )

        max_index_val_acc, max_value_val_acc[i] = max(enumerate(history.history['val_accuracy'],), key=operator.itemgetter(1))
        train_acc[i] = history.history['accuracy'][max_index_val_acc]

        min_index_val_loss, min_value_val_loss[i] = min(enumerate(history.history['val_loss'], ), key=operator.itemgetter(1))
        train_loss[i] = history.history['loss'][min_index_val_loss]


        out_string = f'f_{i}\nval_acc={max_value_val_acc[i]} train_acc={train_acc[i]} ep={max_index_val_acc+1}\nval_loss={min_value_val_loss[i]} train_loss={train_loss[i]} ep={min_index_val_loss + 1}'

        save_results(output_path, out_string)
        i +=1

        tf.keras.backend.clear_session()
        # dnn_model = build_and_compile_model(normalizer,n_neurons,dropout,learning_rate)
        dnn_model = build_and_compile_model(normalizer,n_neurons,dropout,learning_rate)


        dnn_model_pipe = make_pipeline(MinMaxScaler(),dnn_model)

    out_string = f'median\nval_acc={np.median(max_value_val_acc)} train_acc={np.median(train_acc)}\nval_loss={np.median(min_value_val_loss)} train_loss={np.median(train_loss)}'

    save_results(output_path, out_string)


percent of object with nan value and orginals:  0.00, 89.274189


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'